#### Notebook for running Chain-of-Thought with no supporting context experiments

In [1]:
!pwd

/content


In [2]:
colab = True#False#
usingGitHub = True#False#
if(colab == True):
  '''#!pip install -q -U transformers==4.37.2
  #!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0
  !pip install transformers
  !pip install accelerate
  !pip install bitsandbytes'''

  !pip install transformers
  !pip install accelerate
  !pip install bitsandbytes
  !pip install joblib==1.4.2
  !pip install langchain==0.0.162
  !pip install langchain-core==0.2.43
  !pip install numpy==1.26.4
  !pip install openai==0.27.4
  !pip install pandas==1.5.3
  !pip install python-dotenv==1.0.0
  !pip install scikit-learn==1.5.1
  !pip install tenacity==8.2.2
  !pip install tiktoken==0.8.0
  !pip install transformers==4.46.2
  !pip install wikipedia==1.4.0

  if(usingGitHub == True):
    %cd /content
    !git clone https://github.com/DEVdaVinci/Reflexion-LLaVA.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.9/770.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
ERROR: pip's dependency resolver does no

In [3]:
import sys, os

sys.path.append('..')
root = '../root/'
if(colab == True):
  sys.path.append('/content/Reflexion-LLaVA')
  sys.path.append('/content/Reflexion-LLaVA/hotpotqa_runs')
  sys.path.append('/content/drive/MyDrive/Research/Catagories/Image-to-Prompt/Models/reflexion (OG)/')
  %cd /content/drive/MyDrive/Research/Catagories/Image-to-Prompt/Models/reflexion (OG)/hotpotqa_runs/notebooks
  #root = '/content/Reflexion-LLaVA/hotpotqa_runs/root/'

[Errno 2] No such file or directory: '/content/drive/MyDrive/Research/Catagories/Image-to-Prompt/Models/reflexion (OG)/hotpotqa_runs/notebooks'
/content


In [4]:
!ls ..

bin			    dev    lib64		     opt		run   tools
boot			    etc    libx32		     proc		sbin  usr
content			    home   media		     python-apt		srv   var
cuda-keyring_1.0-1_all.deb  lib    mnt			     python-apt.tar.xz	sys
datalab			    lib32  NGC-DL-CONTAINER-LICENSE  root		tmp


In [5]:
if colab:
    %cd /content/Reflexion-LLaVA/hotpotqa_runs
    !pwd

/content/Reflexion-LLaVA/hotpotqa_runs
/content/Reflexion-LLaVA/hotpotqa_runs


In [ ]:
os.environ['OPENAI_API_KEY']= "YOUR_API_KEY"

In [7]:
%ls

agents.py  environment.py  llm.py    notebooks/  react.py          root/     util.py
data/      fewshots.py     mocks.py  prompts.py  requirements.txt  tests.py


In [8]:
from util import summarize_trial, log_trial, save_agents
import joblib
from agents import CoTAgent, ReflexionStrategy

#### Load the HotPotQA Sample

In [9]:

#print(sys.executable)  # Should point to reflexionENV
#print(sys.path)  # Should include paths for reflexionENV packages
#!pip list
#import pandas as pd
#print(pd.__version__)

In [11]:
if usingGitHub:
  hotpot = joblib.load('/content/Reflexion-LLaVA/hotpotqa_runs/data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)
else:
  hotpot = joblib.load('../data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

#### Define the Reflexion Strategy

In [12]:
print(ReflexionStrategy.__doc__)


    NONE: No reflection
    LAST_ATTEMPT: Use last reasoning trace in context 
    REFLEXION: Apply reflexion to the next reasoning trace 
    LAST_ATTEMPT_AND_REFLEXION: Use last reasoning trace in context and apply reflexion to the next reasoning trace 
    


In [13]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

#### Initialize a CoTAgent for each question

In [14]:
from prompts import cot_simple_reflect_agent_prompt, cot_simple_reflect_prompt, cot_simple_agent_prompt
from fewshots import COTQA_SIMPLE6, COT_SIMPLE_REFLECTION

agents = [CoTAgent(question = row['question'],
                   context = '',
                   key = row['answer'],
                   agent_prompt=cot_simple_agent_prompt if strategy == ReflexionStrategy.NONE else cot_simple_reflect_agent_prompt,
                   cot_examples = COTQA_SIMPLE6,
                   reflect_prompt = cot_simple_reflect_prompt,
                   reflect_examples = COT_SIMPLE_REFLECTION,
                      ) for _, row in hotpot.iterrows()]

In [15]:
for _, row in hotpot.iterrows():
    print(f"question:\n\tdata type: {type(row['question'])}\n\tvalue: {row['question']}\n")
    print("-----------\n")
    print(f"Context: (Has been set to an empty string)\n")
    print("-----------\n")
    print(f"key: \n\tdata type: {type(row['answer'])}\n\tvalue: {row['answer']}\n")
    print("-----------\n")
    print(f"agent_prompt: \n\tdata type: {type(cot_simple_agent_prompt if strategy == ReflexionStrategy.NONE else cot_simple_reflect_agent_prompt)}\n\tvalue: {cot_simple_agent_prompt if strategy == ReflexionStrategy.NONE else cot_simple_reflect_agent_prompt}\n")
    print("-----------\n")
    print(f"cot_examples: \n\tdata type: {type(COTQA_SIMPLE6)}\n\tvalue: {COTQA_SIMPLE6}\n")
    print("-----------\n")
    print(f"reflect_prompt: \n\tdata type: {type(cot_simple_reflect_prompt)}\n\tvalue: {cot_simple_reflect_prompt}\n")
    print("-----------\n")
    print(f"reflect_examples: \n\tdata type: {type(COT_SIMPLE_REFLECTION)}\n\tvalue: {COT_SIMPLE_REFLECTION}\n")
    print("\n\n\n\n\n====================================================================================================\n\n\n\n\n")

Streaming output truncated to the last 5000 lines.

Question: Which magazine was started first Arthur's Magazine or First for Women?
Thought: Let's think step by step. Arthur's Magazine was started in 1844. First for Women was started in 1989. 1844 (Arthur's Magazine) < 1989 (First for Women), so Arthur's Magazine was started first.
Action: Finish[Arthur's Magazine]

Question: Were Pavel Urysohn and Leonid Levin known for the same type of work?
Thought: Let's think step by step. Pavel Urysohn is a mathematician. Leonid Levin is a mathematician and computer scientist. So Pavel Urysohn and Leonid Levin have the same type of work.
Action: Finish[Yes]

-----------

reflect_prompt: 
	data type: <class 'langchain.prompts.prompt.PromptTemplate'>
	value: input_variables=['examples', 'question', 'context', 'scratchpad'] output_parser=None partial_variables={} template='You are an advanced reasoning agent that can improve based on self refection. You will be given a previous reasoning trial in w

#### Run `n` trials

In [16]:
n = 5
trial = 0
log = ''
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        agent.run(reflexion_strategy = strategy)
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_trial(agents, trial)
    correct, incorrect = summarize_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}\n\n\n\n\n\n\n\n\n\n')

Thought: Let's think step by step. VIVA Media AG changed its name to VIVA Entertainment in 2004. The new acronym stands for Videoverwertungsanstalt.
Action: Finish[Videoverwertungsanstalt]
Answer: Gesellschaft mit beschränkter Haftung
Thought: Let's think step by step. Jonny Craig has been a member of bands such as Dance Gavin Dance and Emarosa. Pete Doherty has been a member of bands such as The Libertines and Babyshambles. Pete Doherty has been a member of more bands.
Action: Finish[Pete Doherty]
Answer: Jonny" Craig
Thought: Let's think step by step. The Missouri Compromise admitted Missouri as a slave state and Maine as a free state. The first governor after the compromise was from Maine.
Action: Finish[Maine]
Answer: Bath, Maine
Thought: Let's think step by step. The creator of "Wallace and Gromit" is Nick Park. The animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes is "Creature Comforts." So the answer is "Creature Comforts."


KeyboardInterrupt: 

#### Save the result log

In [ ]:
with open(os.path.join(root, 'CoT', 'no_context', strategy.value, f'{len(agents)}_questions_{trial}_trials.txt'), 'w') as f:
    f.write(log)
save_agents(agents, os.path.join(root, 'CoT', 'no_context', strategy.value, 'agents'))